<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/LangChain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial Source: https://www.youtube.com/watch?v=ziu87EXZVUE&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=11

In [ ]:
!pip -q install openai langchain tiktoken python-dotenv faiss-gpu huggingface_hub google-search-results wikipedia


In [26]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('/content/drive/MyDrive/Projects/keys/secrets.json')

# Use variables
openai_api = os.getenv('OPENAI_API_KEY')
serpapi_api = os.getenv('SERPAPI_API_KEY')

# LangChain Agents

### Creating an Agent

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

### Loading Tools

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
# What is a tool like this
tools[1].name, tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [ ]:
tools[1]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<class 'openai.api_resources.completion.Comp

### Initializing the Agent


In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

In [ ]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [ ]:
# Standard LLM Query
agent.run("How are you today?")



> Entering new AgentExecutor chain...
 This is not a question that can be answered with a search or a calculator.
Action: None
Action Input: None
Observation: None is not a valid tool, try one of [Search, Calculator].
Thought: I now know the final answer
Final Answer: I'm doing well, thank you!

> Finished chain.


"I'm doing well, thank you!"

In [ ]:
agent.run("Who is the United States President? What is his current age divided by 2?")



> Entering new AgentExecutor chain...
 I need to find out who the President is and then calculate his age divided by 2.
Action: Search
Action Input: "United States President"
Observation: Joe Biden
Thought: I now need to find out Joe Biden's age
Action: Search
Action Input: "Joe Biden age"
Observation: 80 years
Thought: I now need to divide 80 by 2
Action: Calculator
Action Input: 80/2
Observation: Answer: 40.0
Thought: I now know the final answer
Final Answer: Joe Biden is 80 years old and his age divided by 2 is 40.0.

> Finished chain.


'Joe Biden is 80 years old and his age divided by 2 is 40.0.'

In [ ]:
agent.run("What is the average age in the United States? How much less is that then the age of current US President?")



> Entering new AgentExecutor chain...
 I need to find the average age in the US and then compare it to the age of the current president.
Action: Search
Action Input: Average age in the United States
Observation: ["The U.S. is a country of 50 states covering a vast swath of North America, with Alaska in the northwest and Hawaii extending the nation’s presence into the Pacific Ocean. Major Atlantic Coast cities are New York, a global finance and culture center, and capital Washington, DC. Midwestern metropolis Chicago is known for influential architecture and on the west coast, Los Angeles' Hollywood is famed for filmmaking. ― Google", 'United States type: Country in North America.', 'United States kgmid: /m/09c7w0.', 'United States president: Joe Biden.', 'United States capital: Washington, D.C..', 'United States co2_emissions_per_capita: 14.67 metric tons (2019).', 'United States electricity_consumption_per_capita: 12,993.97 kWh (2014).', 'United States energy_use_per_capita: 6,804.0

'The average age in the United States is 38.9 years, which is 41.1 years less than the age of current US President Joe Biden (80 years).'

# New Agent

In [ ]:
!pip installlangchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
tools = load_tools(["serpapi", "llm-math", "wikipedia", "terminal"], llm=llm)

In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

In [ ]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nterminal: Run shell commands on this Linux machine.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator, Wikipedia, terminal]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input q

In [ ]:
agent.run("Who is the head of DeepMind?")



> Entering new AgentExecutor chain...
 I need to find out who the head of DeepMind is.
Action: Search
Action Input: "head of DeepMind"
Observation: [{'title': 'AI could herald a new era of scientific discovery for Britain', 'link': 'https://www.telegraph.co.uk/business/2023/06/13/ai-scientific-discovery-britain-rishi-sunak/', 'source': 'The Telegraph', 'date': 'Jun 13, 2023', 'thumbnail': 'https://serpapi.com/searches/652810cc8b7ed2c807fbdb1d/images/4892ff4d902aeddeed2c747662c58e622ba07053d0aae3b9.jpeg'}, {'title': 'Discovering faster matrix multiplication algorithms with reinforcement learning', 'link': 'https://www.nature.com/articles/s41586-022-05172-4', 'source': 'Nature', 'date': 'Oct 5, 2022', 'thumbnail': 'https://serpapi.com/searches/652810cc8b7ed2c807fbdb1d/images/4892ff4d902aedde523eec3a6bf44bed77ff136c9fde5f08.jpeg'}, {'title': "It's Not All Fun and Games: How DeepMind Unlocks Medicine's Secrets", 'link': 'https://www.medscape.com/viewarticle/975013', 'source': 'Medscape',

'The head of DeepMind is Demis Hassabis.'

In [ ]:
agent.run("Who is the CEO of the PGA Tour?")



> Entering new AgentExecutor chain...
 I need to find out who the CEO of the PGA Tour is.
Action: Wikipedia
Action Input: PGA Tour
Observation: Page: PGA Tour
Summary: The PGA Tour (stylized in all capital letters as PGA TOUR by its officials) is the organizer of professional golf tours in the United States and North America. It organizes most of the events on the flagship annual series of tournaments also known as the PGA Tour, as well as the PGA Tour Champions (age 50 and older) and the Korn Ferry Tour (for professional players who have not yet qualified to play on the PGA Tour), as well as the PGA Tour Canada, PGA Tour Latinoamérica, and formerly the PGA Tour China. The PGA Tour is a nonprofit organization headquartered in Ponte Vedra Beach, Florida, a suburb southeast of Jacksonville.Originally established by the Professional Golfers' Association of America (PGA of America), it was spun off in December 1968 into a separate organization for tour players, as opposed to club profess

'Jay Monahan is the CEO of the PGA Tour.'

In [ ]:
agent.run("If I square the number for the street address of DeepMind what answer do I get?")



> Entering new AgentExecutor chain...
 I need to find the street address of DeepMind first.
Action: Search
Action Input: "DeepMind street address"
Observation: Where is DeepMind's headquarters? DeepMind's headquarters is located at 5 New Street Square, London.
Thought: I need to use the calculator to square the number.
Action: Calculator
Action Input: 5^2
Observation: Answer: 25
Thought: I now know the final answer.
Final Answer: 25

> Finished chain.


'25'